In [1]:
spark
s3_url = 's3a://dse-cohort5-group5/wildfire_capstone/timeLaggedGridMetFeatures/partitioned/{agg}'
aggs = [
    'min',
    'max',
    'mean',
    'cumulative',
    'std',
]
dfs = []
table_names = []
base_df = spark.read.parquet('s3a://dse-cohort5-group5/wildfire_capstone/integratedData.renamed.parquet.gz')
table_names.append("fire_occurrences")
base_df.createOrReplaceTempView(table_names[0])
dfs.append(base_df)
for agg in aggs:
    df = spark.read.parquet(s3_url.format(agg=agg))
    df.createOrReplaceTempView(agg)
    dfs.append(df)
    table_names.append(agg)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15,application_1586646843498_0016,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
base_df.printSchema()

root
 |-- date: long (nullable = true)
 |-- precipitation_amount_mm: double (nullable = true)
 |-- relative_humidity_%: double (nullable = true)
 |-- specific_humidity_kg/kg: double (nullable = true)
 |-- surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- wind_speed_m/s: double (nullable = true)
 |-- max_air_temperature_K: double (nullable = true)
 |-- min_air_temperature_K: double (nullable = true)
 |-- burning_index_g_Unitless: double (nullable = true)
 |-- dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- energy_release_component-g_Unitless: double (nullable = true)
 |-- potential_evapotranspiration_mm: double (nullable = true)
 |-- mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- fire_occurred: integer (nullable = true)
 |-- acres_burned: double (nullable = true)
 |-- fire_name: strin

In [3]:
dfs[1].printSchema()

root
 |-- 7d_min_precipitation_amount_mm: double (nullable = true)
 |-- 7d_min_relative_humidity_%: double (nullable = true)
 |-- 7d_min_specific_humidity_kg/kg: double (nullable = true)
 |-- 7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2: double (nullable = true)
 |-- 7d_min_wind_from_direction_Degrees_Clockwise_from_north: double (nullable = true)
 |-- 7d_min_wind_speed_m/s: double (nullable = true)
 |-- 7d_min_max_air_temperature_K: double (nullable = true)
 |-- 7d_min_min_air_temperature_K: double (nullable = true)
 |-- 7d_min_burning_index_g_Unitless: double (nullable = true)
 |-- 7d_min_dead_fuel_moisture_100hr_Percent: double (nullable = true)
 |-- 7d_min_dead_fuel_moisture_1000hr_Percent: double (nullable = true)
 |-- 7d_min_energy_release_component-g_Unitless: double (nullable = true)
 |-- 7d_min_potential_evapotranspiration_mm: double (nullable = true)
 |-- 7d_min_mean_vapor_pressure_deficit_kPa: double (nullable = true)
 |-- 14d_min_precipitation_amount_mm: double (n

In [4]:
pca_cols = [table_names[0] + ".`{}`".format(c) for c in base_df.schema.names[1:-6]]
for i, df in enumerate(dfs[1:], 1):
    pca_cols += [table_names[i] + '.`{}`'.format(c) for c in df.schema.names[:-4]]
pca_cols

['fire_occurrences.`precipitation_amount_mm`', 'fire_occurrences.`relative_humidity_%`', 'fire_occurrences.`specific_humidity_kg/kg`', 'fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`', 'fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`', 'fire_occurrences.`wind_speed_m/s`', 'fire_occurrences.`max_air_temperature_K`', 'fire_occurrences.`min_air_temperature_K`', 'fire_occurrences.`burning_index_g_Unitless`', 'fire_occurrences.`dead_fuel_moisture_100hr_Percent`', 'fire_occurrences.`dead_fuel_moisture_1000hr_Percent`', 'fire_occurrences.`energy_release_component-g_Unitless`', 'fire_occurrences.`potential_evapotranspiration_mm`', 'fire_occurrences.`mean_vapor_pressure_deficit_kPa`', 'min.`7d_min_precipitation_amount_mm`', 'min.`7d_min_relative_humidity_%`', 'min.`7d_min_specific_humidity_kg/kg`', 'min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`', 'min.`7d_min_wind_from_direction_Degrees_Clockwise_from_north`', 'min.`7d_min_wind_speed_m/s`',

In [5]:
select_template = """
SELECT fire_occurrences.date, fire_occurrences.latitude, fire_occurrences.longitude, {pca_cols}
FROM fire_occurrences, {tables} WHERE """
where_template = """
to_unix_timestamp({table}.date) = cast(fire_occurrences.date/1e9 as long)
AND {table}.latitude = fire_occurrences.latitude
AND {table}.longitude = fire_occurrences.longitude"""

query = select_template.format(tables=", ".join(aggs),
                               pca_cols=", ".join(pca_cols)) \
        + " AND ".join(where_template.format(table=agg) for agg in aggs)
query

'\nSELECT fire_occurrences.date, fire_occurrences.latitude, fire_occurrences.longitude, fire_occurrences.`precipitation_amount_mm`, fire_occurrences.`relative_humidity_%`, fire_occurrences.`specific_humidity_kg/kg`, fire_occurrences.`surface_downwelling_shortwave_flux_in_air_W_m-2`, fire_occurrences.`wind_from_direction_Degrees_Clockwise_from_north`, fire_occurrences.`wind_speed_m/s`, fire_occurrences.`max_air_temperature_K`, fire_occurrences.`min_air_temperature_K`, fire_occurrences.`burning_index_g_Unitless`, fire_occurrences.`dead_fuel_moisture_100hr_Percent`, fire_occurrences.`dead_fuel_moisture_1000hr_Percent`, fire_occurrences.`energy_release_component-g_Unitless`, fire_occurrences.`potential_evapotranspiration_mm`, fire_occurrences.`mean_vapor_pressure_deficit_kPa`, min.`7d_min_precipitation_amount_mm`, min.`7d_min_relative_humidity_%`, min.`7d_min_specific_humidity_kg/kg`, min.`7d_min_surface_downwelling_shortwave_flux_in_air_W_m-2`, min.`7d_min_wind_from_direction_Degrees_Cloc

In [6]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
assembler = VectorAssembler(inputCols=[c.split('.')[1].replace('`', '') for c in pca_cols], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
joined = spark.sql(query)
joined = assembler.setHandleInvalid("skip").transform(joined)
scalerModel = scaler.fit(joined)
joined = scalerModel.transform(joined)
# joined.cache()
# joined.select('scaledFeatures').show(3, vertical=True)

In [7]:
scalerModel.write().overwrite()\
    .save("s3a://dse-cohort5-group5/wildfire_capstone/scalermodel.sparkobject")

In [8]:
from pyspark.ml.feature import PCA
pca = PCA(k=500, inputCol="scaledFeatures", outputCol="pcaFeatures")
model = pca.fit(joined)
result = model.transform(joined)\
    .select('date', 'latitude', 'longitude', 'pcaFeatures')

In [9]:
result.select("pcaFeatures").show(3, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
model.explainedVariance

DenseVector([0.3134, 0.2013, 0.0769, 0.0522, 0.03, 0.024, 0.0235, 0.0217, 0.0207, 0.0158, 0.0124, 0.0111, 0.0097, 0.0084, 0.0081, 0.0077, 0.0069, 0.0067, 0.0062, 0.0052, 0.005, 0.005, 0.0042, 0.0041, 0.0037, 0.0034, 0.0033, 0.0031, 0.0029, 0.0026, 0.0026, 0.0025, 0.0023, 0.0023, 0.0021, 0.0021, 0.002, 0.0019, 0.0018, 0.0018, 0.0017, 0.0017, 0.0017, 0.0016, 0.0016, 0.0015, 0.0014, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0012, 0.0011, 0.0011, 0.0011, 0.0011, 0.001, 0.001, 0.001, 0.0009, 0.0009, 0.0009, 0.0009, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0008, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0007, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0006, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.00

In [11]:
# Better to partition the result
# result.write.parquet("s3a://dse-cohort5-group5/wildfire_capstone/integratedData.pca.parquet.gz",
#                      mode="overwrite",
#                      compression='gzip')

In [12]:
result.createOrReplaceTempView('result')
with_years = spark.sql("""
SELECT date, latitude, longitude, pcaFeatures, 
    from_unixtime(cast(date/1e9 as long), 'yyyy') as year
FROM result
""")
with_years.show(3, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
with_years.write.parquet("s3a://dse-cohort5-group5/wildfire_capstone/integratedData/pca",
                     mode="overwrite",
                     partitionBy=['year'],
                     compression='gzip')

An error was encountered:
Invalid status code '400' from http://ip-172-31-20-242.ec2.internal:8998/sessions/15/statements/13 with error payload: {"msg":"requirement failed: Session isn't active."}


In [14]:
model.write().overwrite()\
    .save("s3a://dse-cohort5-group5/wildfire_capstone/pcamodel.sparkobject")

An error was encountered:
Session 15 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
20/04/13 19:32:44 INFO TaskSetManager: Starting task 144.0 in stage 67.0 (TID 1381, ip-172-31-29-155.ec2.internal, executor 1, partition 144, RACK_LOCAL, 8302 bytes)
20/04/13 19:32:44 INFO TaskSetManager: Finished task 140.0 in stage 67.0 (TID 1377) in 73730 ms on ip-172-31-29-155.ec2.internal (executor 1) (141/200)
20/04/13 19:33:07 INFO TaskSetManager: Starting task 145.0 in stage 67.0 (TID 1382, ip-172-31-21-122.ec2.internal, executor 2, partition 145, NODE_LOCAL, 8302 bytes)
20/04/13 19:33:07 INFO TaskSetManager: Finished task 141.0 in stage 67.0 (TID 1378) in 73945 ms on ip-172-31-21-122.ec2.internal (executor 2) (142/200)
20/04/13 19:33:45 INFO TaskSetManager: Starting task 146.0 in stage 67.0 (TID 1383, ip-172-31-29-155.ec2.internal, executor 1, partition 146, RACK_LOCAL, 8302 bytes)
20/04/13 19:33:45 INFO TaskSetManager: Finished task 142.0 in stage 67.0 (TID 1379) in 736